In [109]:
import os
import numpy as np 
import pandas as pd
import datetime
import psycopg2



host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com",
database="dalmia_digital",
user="postgres",
password="dalmia123"

def connection_db(host, database, user, password):
    conn = psycopg2.connect(
    host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com",
    database="dalmia_digital",
    user="postgres",
    password="dalmia123")
    sql_query = "SELECT * FROM dalmia_sales.sales_data where billing_date > '2021-04-01' "
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    return df


def create_date_columns(df_list):#pass the dataframe in the list format 
    new_df = pd.DataFrame()
    for i in df_list:
        i['date'] = pd.to_datetime(i['billing_date'])
        i['Month_Year'] = pd.to_datetime(i['date']).dt.strftime('%Y-%m')
        i['Year'] = pd.to_datetime(i['date']).dt.year
        i['Quarter'] = i['date'].dt.to_period('Q')
        new_df = pd.concat([new_df,i], axis=0)

    return new_df

def return_yoy_df(host, database, user, password):
    print(1)
    df = connection_db(host, database, user, password)
    print(2)
    #df['Billing Date'] = df['Billing Date'].apply(timeseries_conversion)
    print(3)
    df_sales = create_date_columns([df])
    print(4)
    df_sales = df_sales[['sold_to_party_code','Month_Year','bill_quantity']]
    print(5)
    df_sales =df_sales[df_sales['sold_to_party_code']!='DONATION']
    df_group_month = df_sales.groupby(['sold_to_party_code','Month_Year']).agg({'bill_quantity': ['sum']}).reset_index()
    print(7)
    df_group_month.columns = ['Sold To Party Code', 'Month Year' , 'Sum']
    pivot_table = pd.pivot_table(df_group_month, values='Sum',columns='Month Year', index = 'Sold To Party Code')
    print(8)
    return pivot_table

In [110]:
pivot_table = return_yoy_df(host, database , user, host)

1


/var/folders/m3/m633mc4s5tsggxm0f81370l80000gn/T/ipykernel_748/3013567095.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


2
3
4
5
7
8


In [1]:
pwd

'/Users/nikhilsingh/Downloads/Dalmia_machine_learning/Nikhil/Experiment_folder'

In [11]:
pivot_table = pivot_table[((pivot_table.index)>=7000000)&((pivot_table.index)<7100000)]

In [12]:
pivot_table

Month Year,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,22.0,20.0,20.0,20.0,20.0,122.5,75.0,90.0,155.0,46.0
7000010,57.0,37.0,NaN,24.0,30.0,NaN,41.0,24.0,25.0,24.0,...,33.8,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,42.0,54.5,45.0,74.0,27.0,NaN,NaN,NaN,NaN,NaN
7000019,12.0,50.0,133.0,104.0,165.0,123.5,NaN,7.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7000022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.0,12.0,24.0,22.0,5.0,21.0,24.0,NaN,29.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7040129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5
7040139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
7040141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0


In [26]:
import os
import numpy as np 
import pandas as pd
import datetime
import psycopg2



host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com",
database="dalmia_digital",
user="postgres",
password="dalmia123"

def connection_db(host, database, user, password):
    conn = psycopg2.connect(
    host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com",
    database="dalmia_digital",
    user="postgres",
    password="dalmia123")
    sql_query = "SELECT * FROM dalmia_sales.dalmia_east_discount_new"
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    return df


def create_date_columns(df_list):#pass the dataframe in the list format 
    new_df = pd.DataFrame()
    for i in df_list:
        i['date'] = pd.to_datetime(i['billing_date'])
        i['Month_Year'] = pd.to_datetime(i['date']).dt.strftime('%Y-%m')
        i['Year'] = pd.to_datetime(i['date']).dt.year
        i['Quarter'] = i['date'].dt.to_period('Q')
        new_df = pd.concat([new_df,i], axis=0)

    return new_df

def return_yoy_df(host, database, user, password):
    print(1)
    df = connection_db(host, database, user, password)
    print(2)
    #df['Billing Date'] = df['Billing Date'].apply(timeseries_conversion)
    print(3)
    df_sales = create_date_columns([df])
    print(4)
    df_sales = df_sales[['sold_to_party_code','Month_Year','bill_quantity']]
    print(5)
    df_sales =df_sales[df_sales['sold_to_party_code']!='DONATION']
    df_group_month = df_sales.groupby(['sold_to_party_code','Month_Year']).agg({'bill_quantity': ['sum']}).reset_index()
    print(7)
    df_group_month.columns = ['Sold To Party Code', 'Month Year' , 'Sum']
    pivot_table = pd.pivot_table(df_group_month, values='Sum',columns='Month Year', index = 'Sold To Party Code')
    print(8)
    return pivot_table

In [27]:
df = connection_db(host, database, user, password)

/var/folders/m3/m633mc4s5tsggxm0f81370l80000gn/T/ipykernel_748/256149962.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393512 entries, 0 to 393511
Data columns (total 60 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   region                                            393512 non-null  object 
 1   state                                             393512 non-null  object 
 2   sales_zone                                        393512 non-null  object 
 3   district                                          393512 non-null  object 
 4   duration                                          393512 non-null  object 
 5   month                                             393512 non-null  object 
 6   customer_code                                     393512 non-null  int64  
 7   cust_name                                         393512 non-null  object 
 8   effected_quantity_cash_discount                   43695 non-null   float64
 9   amou

In [55]:
def connection_db(host, database, user, password):
    conn = psycopg2.connect(
    host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com",
    database="dalmia_digital",
    user="postgres",
    password="dalmia123")
    sql_query = "SELECT * FROM dalmia_sales.dalmia_ne_discount_new"
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    return df

In [56]:
df_ne = connection_db(host, database, user, password)

/var/folders/m3/m633mc4s5tsggxm0f81370l80000gn/T/ipykernel_748/328377487.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


In [57]:
df_ne.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15282 entries, 0 to 15281
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   sales_zone                         15282 non-null  object 
 1   duration                           15282 non-null  object 
 2   month                              15282 non-null  object 
 3   cust_code                          15282 non-null  float64
 4   cust_name                          15282 non-null  object 
 5   cash_discountqtymt                 15282 non-null  float64
 6   amtrs                              15282 non-null  float64
 7   flat_discountqtymt                 15282 non-null  float64
 8   flat_discountamtrs                 15282 non-null  float64
 9   quantity_discountqtymt             15282 non-null  float64
 10  quantity_discountamtrs             15282 non-null  float64
 11  target_discountqtymt               15282 non-null  flo

In [29]:
df = df[['customer_code','amount_total', 'month']]
df = df

In [31]:
df_discount_month = df.groupby(['customer_code','month']).agg({'amount_total': ['sum']}).reset_index()
    
df_discount_month.columns = ['Sold To Party Code', 'month' , 'Sum']
pivot_table_discount = pd.pivot_table(df_discount_month, values='Sum',columns='month', index = 'Sold To Party Code')

In [35]:
pivot_table_discount = pivot_table_discount[["Apr'22" ,"May'22", "Jun'22" ,"Jul'22" ,"Aug'22" ,"Sep'22" ,"Oct'22" ,"Nov'22" ,"Dec'22","Jan'23" ,"Feb'23" ,"Mar'23" ]]

In [37]:
pivot_table_discount = pivot_table_discount[(pivot_table_discount.index>=7000000)&(pivot_table_discount.index<7100000)]

In [38]:
pivot_table_discount

month,Apr'22,May'22,Jun'22,Jul'22,Aug'22,Sep'22,Oct'22,Nov'22,Dec'22,Jan'23,Feb'23,Mar'23
Sold To Party Code,,,,,,,,,,,,
7000004,57330.58,52083.4075,12564.7000,11481.9100,11250.0000,4800.0000,5750.0000,9688.0000,58952.5200,21094.7800,18927.0,144323.0
7000010,17500.00,7830.6200,NaN,17294.7300,6986.2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7000015,17499.34,47316.5125,53150.8263,27085.0453,38413.8825,20554.6675,12238.4578,17762.3125,NaN,15070.0000,NaN,NaN
7000022,NaN,1086.0938,707.3263,4375.0781,5821.3594,10897.4800,4692.6250,1420.4375,11337.7891,4939.1800,NaN,2635.0
7000025,40709.34,43750.0000,67296.2000,102617.1325,41882.8125,38189.0625,9165.0000,56590.0000,24722.4375,39260.8828,26684.0,88732.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7039507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209.0
7039512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1483.5
7039519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190.0


In [66]:
from utility.datetime_util import timeseries_conversion
def create_date_columns(df_list):#pass the dataframe in the list format 
    new_df = pd.DataFrame()
    for i in df_list:
        #i['date'] = pd.to_datetime(i['month'])
        i['Month_Year'] = pd.to_datetime(i['month']).dt.strftime('%Y-%m')
        #i['Year'] = pd.to_datetime(i['date']).dt.year
        #i['Quarter'] = i['date'].dt.to_period('Q')
        new_df = pd.concat([new_df,i], axis=0)

    return new_df

df_ne = df_ne[['cust_code','subhyatratotal_amt', 'month']]
df_ne = df_ne[df_ne['month'].isin(['44927', '44652', '44682', '44713','44743', '44774','44805', '44835',
       '44866', '44896', '44986', '44958'])]


In [67]:
df_ne['month'] = df_ne['month'].astype(int)

In [68]:
df_ne['month'] = df_ne['month'].apply(timeseries_conversion)
df_ne = create_date_columns([df_ne])
df_ne = df_ne[['cust_code','subhyatratotal_amt', 'Month_Year']]

In [70]:
df_ne['Month_Year'].unique()

array(['2023-01', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08',
       '2022-09', '2022-10', '2022-11', '2022-12', '2023-03', '2023-02'],
      dtype=object)

In [71]:
df_discount_month_ne = df_ne.groupby(['cust_code','Month_Year']).agg({'subhyatratotal_amt': ['sum']}).reset_index()

    
df_discount_month_ne.columns = ['Sold To Party Code', 'month' , 'Sum']
pivot_table_discount_ne = pd.pivot_table(df_discount_month_ne, values='Sum',columns='month', index = 'Sold To Party Code')

In [90]:
pivot_table_discount_ne.index = pivot_table_discount_ne.index.astype(int)

In [105]:
pivot_table_discount_ne.columns = ["Apr'22", "May'22", "Jun'22", "Jul'22", "Aug'22", "Sep'22", "Oct'22",
       "Nov'22", "Dec'22", "Jan'23", "Feb'23", "Mar'23"]

In [75]:
def connection_db(host, database, user, password):
    conn = psycopg2.connect(
    host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com",
    database="dalmia_digital",
    user="postgres",
    password="dalmia123")
    sql_query = "SELECT * FROM dalmia_sales.dalmia_south_discount"
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    return df

In [76]:
df_south = connection_db(host, database, user, password)

/var/folders/m3/m633mc4s5tsggxm0f81370l80000gn/T/ipykernel_748/2079374896.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


In [79]:
df_south = df_south[['cust_code', 'month' , 'total_amt']]
df_south

,cust_code,month,total_amt
0,7014021,Apr'22,782.00
1,7014023,Apr'22,2983.60
2,7014032,Apr'22,354550.00
3,7014163,Apr'22,24423.36
4,7014170,Apr'22,8386.74
...,...,...,...
187905,7037879,Dec'22,300.00
187906,7037880,Dec'22,150.00
187907,7038750,Dec'22,150.00
187908,7038752,Dec'22,75.00


In [80]:
df_discount_month_south = df_south.groupby(['cust_code','month']).agg({'total_amt': ['sum']}).reset_index()   
df_discount_month_south.columns = ['Sold To Party Code', 'month' , 'Sum']
pivot_table_discount_south = pd.pivot_table(df_discount_month_south, values='Sum',columns='month', index = 'Sold To Party Code')

In [83]:
pivot_table_discount_south= pivot_table_discount_south[["Apr'22" ,"May'22", "Jun'22" ,"Jul'22" ,"Aug'22" ,"Sep'22" ,"Oct'22" ,"Nov'22" ,"Dec'22" ]]

In [103]:
pivot_table_discount.columns


Index(['Apr'22', 'May'22', 'Jun'22', 'Jul'22', 'Aug'22', 'Sep'22', 'Oct'22',
       'Nov'22', 'Dec'22', 'Jan'23', 'Feb'23', 'Mar'23'],
      dtype='object', name='month')

In [85]:
df_playbook = pd.read_excel('data/Dealer Playbook_Working.xlsx')

In [86]:
df_playbook

,Dlr ID,Region,Sales State,TSM,ASO Name (Id),Sold To Party Id,Sold-to Cust,Sold To District,Target Vol MT,Sales Vol MT,...,100-105%,95-100%,90-95%,80-90%,70-80%,60-70%,50-60%,40-50%,25-40%,0-25%
0,7039488,NE,NE3,Rajdeep Singh,Kerdalis Kharlyngdoh | EMP1036527,7039488,D KHYLLEP ENTERPRISE,Meghalaya,45,68.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7035026,WEST,MH1,Sachin Malviya,Prashant Namdevrao Pimpare | EMP1041398,7035026,Haji Traders,Yavatmal,190,52.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,7035089,EAST,BHN,Rahul Pandey,Subash Gupta | EMP0006661,7035089,MEERA AUTOMOBILE TISCON,VAISHALI,136,37.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,7037292,EAST,BHN,Mukesh Rajbher,Surendra Kumar Pathak | EMP1035393,7037292,MUNNA ENTERPRISES,DARBHANGA,120,15.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,7036539,EAST,BHN,Mukesh Rajbher,Surendra Kumar Pathak | EMP1035393,7036539,Santosh kumar jha,DARBHANGA,32,18.50,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9469,7038920,SOUTH,TN3,Victor R,Krishnamoorthy P | EMP1033993,7038920,ZION HARDWARES,Kanyakumari,11,6.00,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
9470,7038390,CENTRAL,UP2,_NA_,_NA_ | EMP1037528,7038390,ZIYA TRADING COMPANY,Bahraich,0,27.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9471,7035994,NE,NE3,Dorendrajit Nongmeikapam,Saikat Dhar | EMP0009290,7035994,ZR ENTERPRISE,Mizoram,55,58.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9472,7036824,NE,NE3,Dorendrajit Nongmeikapam,Saikat Dhar | EMP0009290,7036824,ZUALTEA BUSINESS CENTRE,Mizoram,220,147.00,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [87]:
df_playbook.index = df_playbook['Dlr ID']
df_playbook.drop('Dlr ID', axis =1 , inplace = True)

In [96]:
df_playbook_east = df_playbook[df_playbook['Region']=='EAST']

In [97]:
df_east = df_playbook_east.merge(pivot_table_discount , left_index=True , right_index=True , how = 'left')

In [106]:
df_east = df_east.merge(pivot_table , left_index=True , right_index=True , how = 'left')

In [108]:
df_east.to_excel('data/east_discount_sales.xlsx')